In [2]:
import os
import subprocess
import warnings
import json
import calendar
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np

from scipy.spatial.distance import cdist
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [3]:
# KMeans Class Declaration
class KMeans(object):
    def __init__(self, k=8, distance = 'euclid'):
        self.k = k
        if (distance == 'euclid'):
            self._distance = 'euclidean'
        elif (distance == 'haversine'):
            self._distance = self._distance_haversine
        elif (distance == 'manhattan'):
            self._distance = self._distance_manhattan
        elif (distance == 'street'):
            self._distance = self._distance_street
        else:
            raise Exception('Invalid distance metric')
    
    def _step(self):
        """Compute distance, assign groups, recompute centers"""
        distance = cdist(self.X,self.cluster_centers,metric=self._distance)
        self.labels = distance.argmin(1)
       # centers = np.zeros((self.k,2))
        for cluster in range(self.k):
            points = self.X[self.labels == cluster]
            if len(points) == 0:
                distance = cdist(self.X,np.delete(self.cluster_centers,cluster,0),metric=self._distance)
                mean_dist = np.mean(distance,0)
                self.cluster_centers[cluster] = mean_dist.argmax()
            else:
                self.cluster_centers[cluster] = np.mean(points,0)
       # self.cluster_centers = centers
        
    def _distance_haversine(self,a,b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        d_lat = lat_2 - lat_1
        d_lon = lon_2 - lon_1
        
        arc = np.sin(d_lat/2.0)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(d_lon/2)**2
        
        c = 2 * np.arcsin(np.sqrt(arc))
        km = 6372.8 * c
        return km
    
    def _distance_manhattan(self, a, b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        manhattanDistance = abs(lat_2 - lat_1) + abs(lon_2 - lon_1)
        return manhattanDistance
    
    def _distance_street(self, a, b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        
        query = 'http://localhost:5000/route/v1/driving/%s,%s;%s,%s?overview=false' % (str(lon_1), str(loat_1), str(lon_2), str(lat_2))
        result = subprocess.run(["curl", query], stdout=subprocess.PIPE)
        resultStr = result.stdout.decode("utf-8")
        resultDict = json.loads(resultStr)
        streetDistance = resultDict['routes'][0]['distance']
        
        return streetDistance
   
    def _init_centers(self, X):
        unique = np.unique(X, axis=0)
        index = np.random.permutation(len(unique))[:self.k]
        return unique[index]
    
    def fit(self,X, centers = None):
        '''Expects centers to be inputted, if not random'''
        self.labels = np.zeros(len(X))
        self.X = X
        if centers is not None:
            self.cluster_centers = centers 
        else:
            self.cluster_centers = self._init_centers(X)
        old_centers = np.zeros((self.k,2))
    #    self.i = 0
        while(not np.array_equal(old_centers, self.cluster_centers)):
            old_centers = self.cluster_centers.copy()
            self._step()
         #   self.i+=1

In [12]:
def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def point_similarity(X,geo_labels, euc_labels,k):
    '''For an inputted series of points, geodesic labels, euclidean labels, and k-value
       returns the point-similarity index per geodesic cluster
    '''

    euc_cluster_totals = np.zeros(k,dtype=np.int)
    geo_euc_composition = [np.zeros(k,dtype=np.int)* 1 for i in range(k)]
    
    for index,point in enumerate(geo_labels):
        euc_cluster_totals[euc_labels[index]] += 1
        geo_euc_composition[point][euc_labels[index]] += 1
    
    point_sim = []
    for geo_cluster in range(k):
        sim = 0
        for euc_cluster in range(k):
            matching_points = geo_euc_composition[geo_cluster][euc_cluster]
            euc_percentage = matching_points / euc_cluster_totals[euc_cluster]
            geo_percentage = matching_points / np.sum(geo_euc_composition[geo_cluster])
            sim += euc_percentage * geo_percentage
        point_sim.append(sim)

    return np.array(point_sim)

def percent_similarity(a,b):
    return len(a[a==b])/len(a)

def minority_probability(X,cluster_number,geo_labels,demographics):
        points = X[geo_labels == cluster_number]
        # geoJSON puts points in Long/Lat order
        # but points are in lat/long earlier
        hull = shapely.geometry.multipoint.MultiPoint([[p[1],p[0]] for p in points]).convex_hull
  
        pop = np.zeros(7)
        for index in range(len(demographics)):
            census_tract = demographics.loc[index,'geometry']
            intersect = hull.intersection(census_tract)
            overlap = intersect.area/census_tract.area
            if (overlap != 0):
                pop = pop + (np.array(demographics.loc[index,['White','Black or African American', 'American Indian and Ala Native',
                   'Asian','Native Hawaiian/other Pac Isl', 'Multiple Race',
                   'Other Race']]) * overlap)
        
        if (np.all(pop ==0)):
            return 0
        
        return (pop[1:]/np.sum(pop)).sum()

def bias_index(X, geo_labels, euc_labels, demographics, k):
    if np.all(geo_labels == euc_labels):
        return 0

    dissimilarity_index = 1 - point_similarity(X,geo_labels,euc_labels,k)
    minority_prob = np.array([minority_probability(X,cluster,geo_labels,demographics) 
                              for cluster in range(k)])
    
    potential_bias = minority_prob * dissimilarity_index
    return potential_bias.mean()


In [21]:
def create_bias_matrix():
    columns = ['year', 'k']
    for crime in ['theft', 'mtheft', 'assault', 'robbery']:
        for month in calendar.month_abbr:
            if month != "":
                columns.append(crime + "_" + month.lower())

    frame_list = []
    for year in range(2005,2017):
        for k in range(2,11):
            year_list = [str(year)]
            year_list.append(k)
            year_list.extend([np.nan for i in range(48)])
            frame_list.append(year_list)

    bias_frame = pd.DataFrame(data=frame_list, columns=columns)

    return bias_frame
    

In [32]:
# update the bias matrix with one value
def store_bias(bias_value, bias_frame, year, monthNum, k, crime):
    year_array = np.array(bias_frame.year == str(year)) 
    k_array = np.array(bias_frame.k == k)
    row_index = np.logical_and(year_array,k_array)
    col_name = crime + "_" + calendar.month_abbr[monthNum].lower()

    bias_frame.at[row_index, col_name] = bias_value
    return bias_frame

In [33]:
# global variables
demographics = gpd.read_file('../resources/census.geoJSON')
dataDestination = '../data/clustered_data/'
dataSource = '../data/raw_data/'
bias_frame = create_bias_matrix()

rawDataList = os.listdir(dataSource)
rawDataList = [file for file in rawDataList if file.endswith('.csv')]
minK = 2
maxK = 10

In [34]:
for file in rawDataList:
    df = pd.read_csv(dataSource + file, sep =';')
    [year, monthNum, crime] = file[:-4].split("_")
    year = int(year)
    monthNum = int(monthNum)

    x = []
    y = []

    df['Points'] = df['Location'].apply(gen_coords)
    points = [Point(xy) for xy in zip(x,y)]
    crs = {'init': 'epsg:4326'}
    geo_df = GeoDataFrame(df,crs=crs, geometry=points)
    
    result = geo_df.copy()
    test_list = []

    for index in range(len(result)):
        test_list.append(df.loc[index, 'Points'])

    X = np.array(test_list)

    for k in range(minK ,maxK + 1):
        euclid = KMeans(k = k, distance = 'euclid')
        geodesic = KMeans(k = k, distance = 'haversine')
        manhattan = KMeans(k = k, distance = 'manhattan')

        centers = geodesic._init_centers(X)

        euclid.fit(X, centers = centers)
        geodesic.fit(X, centers = centers) 
        manhattan.fit(X, centers = centers)

        bias_GeodesicEuclid = bias_index(X, geodesic.labels, euclid.labels, demographics, k)
        bias_ManhattanGeodesic = bias_index(X, manhattan.labels, geodesic.labels, demographics, k)
        bias_ManhattanEuclid = bias_index(X, manhattan.labels, euclid.labels, demographics, k)

        bias_frame = store_bias(bias_GeodesicEuclid, bias_frame, year, monthNum, k, crime)
        bias_frame = store_bias(bias_ManhattanGeodesic, bias_frame, year, monthNum, k, crime)
        bias_frame = store_bias(bias_ManhattanEuclid, bias_frame, year, monthNum, k, crime)

        result.loc[:,'e_cluster' + 'K' + str(k)] = euclid.labels.copy()
        result.loc[:,'g_cluster' + 'K' + str(k)] = geodesic.labels.copy()
        result.loc[:,'m_cluster' + 'K' + str(k)] = manhattan.labels.copy()

    result = result.drop('Points', axis=1)

    resultFileName = dataDestination + file[:-4] + "_clustered.js"
    try:
        os.remove(resultFileName)
    except FileNotFoundError:
        pass

    result.to_file(resultFileName, driver='GeoJSON')
    print(resultFileName)


../data/clustered_data/2005_01_assault_clustered.js
../data/clustered_data/2005_01_mtheft_clustered.js
../data/clustered_data/2005_01_robbery_clustered.js
../data/clustered_data/2005_01_theft_clustered.js
../data/clustered_data/2005_02_assault_clustered.js
../data/clustered_data/2005_02_mtheft_clustered.js
../data/clustered_data/2005_02_robbery_clustered.js
../data/clustered_data/2005_02_theft_clustered.js
../data/clustered_data/2005_03_assault_clustered.js
../data/clustered_data/2005_03_mtheft_clustered.js
../data/clustered_data/2005_03_robbery_clustered.js
../data/clustered_data/2005_03_theft_clustered.js
../data/clustered_data/2005_04_assault_clustered.js
../data/clustered_data/2005_04_mtheft_clustered.js
../data/clustered_data/2005_04_robbery_clustered.js
../data/clustered_data/2005_04_theft_clustered.js
../data/clustered_data/2005_05_assault_clustered.js
../data/clustered_data/2005_05_mtheft_clustered.js
../data/clustered_data/2005_05_robbery_clustered.js
../data/clustered_data/20

../data/clustered_data/2008_05_assault_clustered.js
../data/clustered_data/2008_05_mtheft_clustered.js
../data/clustered_data/2008_05_robbery_clustered.js
../data/clustered_data/2008_05_theft_clustered.js
../data/clustered_data/2008_06_assault_clustered.js
../data/clustered_data/2008_06_mtheft_clustered.js
../data/clustered_data/2008_06_robbery_clustered.js
../data/clustered_data/2008_06_theft_clustered.js
../data/clustered_data/2008_07_assault_clustered.js
../data/clustered_data/2008_07_mtheft_clustered.js
../data/clustered_data/2008_07_robbery_clustered.js
../data/clustered_data/2008_07_theft_clustered.js
../data/clustered_data/2008_08_assault_clustered.js
../data/clustered_data/2008_08_mtheft_clustered.js
../data/clustered_data/2008_08_robbery_clustered.js
../data/clustered_data/2008_08_theft_clustered.js
../data/clustered_data/2008_09_assault_clustered.js
../data/clustered_data/2008_09_mtheft_clustered.js
../data/clustered_data/2008_09_robbery_clustered.js
../data/clustered_data/20

../data/clustered_data/2011_09_assault_clustered.js
../data/clustered_data/2011_09_mtheft_clustered.js
../data/clustered_data/2011_09_robbery_clustered.js
../data/clustered_data/2011_09_theft_clustered.js
../data/clustered_data/2011_10_assault_clustered.js
../data/clustered_data/2011_10_mtheft_clustered.js
../data/clustered_data/2011_10_robbery_clustered.js
../data/clustered_data/2011_10_theft_clustered.js
../data/clustered_data/2011_11_assault_clustered.js
../data/clustered_data/2011_11_mtheft_clustered.js
../data/clustered_data/2011_11_robbery_clustered.js
../data/clustered_data/2011_11_theft_clustered.js
../data/clustered_data/2011_12_assault_clustered.js
../data/clustered_data/2011_12_mtheft_clustered.js
../data/clustered_data/2011_12_robbery_clustered.js
../data/clustered_data/2011_12_theft_clustered.js
../data/clustered_data/2012_01_assault_clustered.js
../data/clustered_data/2012_01_mtheft_clustered.js
../data/clustered_data/2012_01_robbery_clustered.js
../data/clustered_data/20

../data/clustered_data/2015_01_assault_clustered.js
../data/clustered_data/2015_01_mtheft_clustered.js
../data/clustered_data/2015_01_robbery_clustered.js
../data/clustered_data/2015_01_theft_clustered.js
../data/clustered_data/2015_02_assault_clustered.js
../data/clustered_data/2015_02_mtheft_clustered.js
../data/clustered_data/2015_02_robbery_clustered.js
../data/clustered_data/2015_02_theft_clustered.js
../data/clustered_data/2015_03_assault_clustered.js
../data/clustered_data/2015_03_mtheft_clustered.js
../data/clustered_data/2015_03_robbery_clustered.js
../data/clustered_data/2015_03_theft_clustered.js
../data/clustered_data/2015_04_assault_clustered.js
../data/clustered_data/2015_04_mtheft_clustered.js
../data/clustered_data/2015_04_robbery_clustered.js
../data/clustered_data/2015_04_theft_clustered.js
../data/clustered_data/2015_05_assault_clustered.js
../data/clustered_data/2015_05_mtheft_clustered.js
../data/clustered_data/2015_05_robbery_clustered.js
../data/clustered_data/20

In [43]:
biasFile = '../data/bias_data/bias.js'
biasJSON = bias_frame.to_json(orient='records')
with open(biasFile,'w') as w:
    w.write('var bias_data =' + biasJSON + ';')

In [9]:
#fix .js files to have var names
path = "../data/clustered_data/"
allFiles = os.listdir(path)

for file in allFiles:
    fileParts = file.split("_")
    varName = fileParts[2] + "_" + fileParts[0] + "_" + fileParts[1]
    print(varName)
    data = 'var ' + varName + ' = ['
    reader = open(path + file, 'r')
    data += (reader.read() + ',')
    data += '];'
    reader.close
    
    writer = open(path + file, 'w')
    writer.write(data)
    writer.close()

mtheft_2005_01
robbery_2005_01
theft_2005_01
assault_2005_02
mtheft_2005_02
robbery_2005_02
theft_2005_02
assault_2005_03
mtheft_2005_03
robbery_2005_03
theft_2005_03
assault_2005_04
mtheft_2005_04
robbery_2005_04
theft_2005_04
assault_2005_05
mtheft_2005_05
robbery_2005_05
theft_2005_05
assault_2005_06
mtheft_2005_06
robbery_2005_06
theft_2005_06
assault_2005_07
mtheft_2005_07
robbery_2005_07
theft_2005_07
assault_2005_08
mtheft_2005_08
robbery_2005_08
theft_2005_08
assault_2005_09
mtheft_2005_09
robbery_2005_09
theft_2005_09
assault_2005_10
mtheft_2005_10
robbery_2005_10
theft_2005_10
assault_2005_11
mtheft_2005_11
robbery_2005_11
theft_2005_11
assault_2005_12
mtheft_2005_12
robbery_2005_12
theft_2005_12
assault_2006_01
mtheft_2006_01
robbery_2006_01
theft_2006_01
assault_2006_02
mtheft_2006_02
robbery_2006_02
theft_2006_02
assault_2006_03
mtheft_2006_03
robbery_2006_03
theft_2006_03
assault_2006_04
mtheft_2006_04
robbery_2006_04
theft_2006_04
assault_2006_05
mtheft_2006_05
robbery_2

mtheft_2016_05
robbery_2016_05
theft_2016_05
assault_2016_06
mtheft_2016_06
robbery_2016_06
theft_2016_06
assault_2016_07
mtheft_2016_07
robbery_2016_07
theft_2016_07
assault_2016_08
mtheft_2016_08
robbery_2016_08
theft_2016_08
assault_2016_09
mtheft_2016_09
robbery_2016_09
theft_2016_09
assault_2016_10
mtheft_2016_10
robbery_2016_10
theft_2016_10
assault_2016_11
mtheft_2016_11
robbery_2016_11
theft_2016_11
assault_2016_12
mtheft_2016_12
robbery_2016_12
theft_2016_12
